In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
from IPython.display import display, display_pretty, Javascript, HTML

## Townhall data

In [1]:
filename = '2016_US_State_Level_Presidential_Results.csv'

Run the next 2 cells if you have the CSV file already.

In [4]:
townhall = pd.read_csv(filename)

In [5]:
del townhall['Unnamed: 0']
townhall.head()

,state_abbr,candidate_name,party,votes_total
0,AL,Donald Trump,GOP,1306925.0
1,AL,Hillary Clinton,DEM,718084.0
2,AL,Gary Johnson,LIB,43869.0
3,AL,Jill Stein,GRN,9287.0
4,AK,Donald Trump,GOP,130415.0


Run the next 4 cells to generate the CSV file data if you don't have it.

In [2]:
# each page has a summary table that rolls up results at the state level
# use it
def cond(x):
    if x:
        return x.startswith("table ec-table") and "table ec-table ec-table-summary" in x
    else:
        return False

In [3]:
# list of state abbreviations
states = ['AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

# headers for csv export
data = [['state_abbr', 'candidate_name', 'party', 'votes_total']]

In [4]:
# loop through each state's web page http://townhall.com/election/2016/president/%s/county, where %s is the state abbr
for state in states:
    r = urllib.request.urlopen('http://townhall.com/election/2016/president/' + state + '/county').read()
    soup = BeautifulSoup(r, "html.parser")

    # loop through each <table> tag with .ec-table class
    tables = soup.findAll('table', attrs={'class':cond})

    for table in tables:
        if table.findParent("table") is None:
            table_body = table.find('tbody')

            rows = table_body.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                # first tbody tr has four td
                # strip text from each td
                divs = cols[0].find_all('div')
                candidate_name = cols[0].text.strip()
                party = cols[1]['class'][0]
                total_votes = int(cols[1].text.strip().replace(',','').replace('-','0'))
                    
                #combine each row's results
                rowData = [state, candidate_name, party, total_votes]
                data.append(rowData)

In [5]:
townhall = pd.DataFrame(data) # throw results in dataframe
new_header = townhall.iloc[0] #grab the first row for the header
townhall = townhall[1:] #take the data less the header row
townhall.columns = new_header #set the header row as the df header
townhall['votes_total'] = townhall['votes_total'].astype('float64')
print(townhall.shape[0])
townhall.head()

235


,state_abbr,candidate_name,party,votes_total
1,AL,Donald Trump,GOP,1306925.0
2,AL,Hillary Clinton,DEM,718084.0
3,AL,Gary Johnson,IND,43869.0
4,AL,Jill Stein,IND,9287.0
5,AK,Donald Trump,GOP,130415.0


Run all the rest to work with the townhall.com data.

In [6]:
# view by state
townhall[(townhall['state_abbr'] == 'ND')]

,state_abbr,candidate_name,party,votes_total
154,ND,Donald Trump,GOP,216133.0
155,ND,Hillary Clinton,DEM,93526.0
156,ND,Gary Johnson,LIB,21351.0
157,ND,Jill Stein,GRN,3769.0
158,ND,Other,OTH,2189.0


In [8]:
# fix third party candidates
townhall.loc[townhall['candidate_name'] =='Jill Stein', 'party'] = 'GRN'
townhall.loc[townhall['candidate_name'] == 'Gary Johnson', 'party'] = 'LIB'
print(townhall[(townhall['candidate_name'] == 'Jill Stein') | (townhall['candidate_name'] == 'Gary Johnson')])

0   state_abbr candidate_name party  votes_total
3           AL   Gary Johnson   LIB      43869.0
4           AL     Jill Stein   GRN       9287.0
7           AK   Gary Johnson   LIB      14593.0
8           AK     Jill Stein   GRN       4445.0
12          AZ   Gary Johnson   LIB      80151.0
13          AZ     Jill Stein   GRN      25255.0
16          AR   Gary Johnson   LIB      29518.0
18          AR     Jill Stein   GRN       9837.0
21          CA   Gary Johnson   LIB     402406.0
22          CA     Jill Stein   GRN     220312.0
26          CO   Gary Johnson   LIB     129451.0
27          CO     Jill Stein   GRN      33147.0
32          CT   Gary Johnson   LIB      48051.0
33          CT     Jill Stein   GRN      22793.0
36          DC   Gary Johnson   LIB       4501.0
37          DC     Jill Stein   GRN       3995.0
40          DE   Gary Johnson   LIB      14751.0
41          DE     Jill Stein   GRN       6100.0
44          FL   Gary Johnson   LIB     206007.0
45          FL     J

In [7]:
party_pivot = pd.pivot_table(data=townhall, index=['state_abbr'], values=['votes_total'], columns=['party'],
                             fill_value=0.0, aggfunc=np.sum)

In [8]:
eff = party_pivot['votes_total']
eff.loc[:,'total'] = eff.sum(axis=1)
eff.loc[:,'dem_waste'] = (eff['DEM'] - eff['GOP'] - 1.0).where(eff['DEM'] > eff['GOP'], eff['DEM'])
eff.loc[:,'gop_waste'] = (eff['GOP'] - eff['GOP'] - 1.0).where(eff['DEM'] < eff['GOP'], eff['GOP'])

/home/dale/miniconda3/envs/analysis/lib/python3.5/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dale/miniconda3/envs/analysis/lib/python3.5/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
results={'total_votes': eff['total'].sum(), 'wasted_gop_votes': eff['gop_waste'].sum(),
         'wasted_dem_votes': eff['dem_waste'].sum()}
results['efficiency_gap'] = (results['wasted_dem_votes']-results['wasted_gop_votes'])/results['total_votes']
results

{'efficiency_gap': 0.15757208192775377,
 'total_votes': 130165463,
 'wasted_dem_votes': 41516759.0,
 'wasted_gop_votes': 21006316.0}

## Export data

In [12]:
townhall.to_csv(filename,sep=',')